In [1]:
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import numpy as np
from fbprophet import Prophet
from functools import partial
import time
from multiprocessing import cpu_count
from joblib import Parallel, delayed, parallel_backend
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")

Importing plotly failed. Interactive plots will not work.


In [2]:
class HelperFunctions():
    def getStores(self, data):
        grouped_by_store = data.groupby('store_id')

        stores = {}
        for key, item in grouped_by_store:
            stores[key] = grouped_by_store.get_group(key)

        return stores
    
    def train_test_split(self, df, year=2016):
        train, test = df[df['year'] < year], df[df['year'] >= year]
        return train, test

In [3]:
class Evaluator():        
    def __init__(self, labelCol, predictionCol):
        self.labelCol = labelCol
        self.predictionCol = predictionCol
    
    def Mape(self, data):        
        return np.mean(abs((data[self.labelCol] - data[self.predictionCol]) / data[self.labelCol]))

In [4]:
class ProphetModel():
    space = {
        'changepoint_prior_scale':hp.choice('changepoint_prior_scale',np.arange(.1,2,.1)),        
        'n_changepoints' : hp.choice('n_changepoints',np.arange(20,200,20))
    }    
        
    def train(self, train, validation, params):
        if params is None:
            prophet = Prophet()
        else:
            prophet = Prophet(**params)

        prophetModel = prophet.fit(train, iter=3000)

        validation_result = validation[["store_id", "year", "month", "y"]]
        validation_result = validation_result.reset_index(drop=True)
        validation_result["yhat"] = prophetModel.predict(validation[['ds']])[["yhat"]]
        
        evaluator = Evaluator(labelCol="y", predictionCol="yhat")
        score = evaluator.Mape(validation_result)    
        
        print('score: {0} model: {1}'.format(score, 'Prophet'))
        return {'loss': score, 'status': STATUS_OK}
    
    def fit(self, data, labelCol):
        df = data.copy()
        df = df.rename(columns={labelCol: 'y'})    
        data = HelperFunctions()
        train, validation = data.train_test_split(df, 2015)
        train = train[['ds','y']]

        trials = Trials()
        bestParams = fmin(partial(self.train, train, validation),
                    space=ProphetModel.space,
                    algo=tpe.suggest,
                    max_evals=1,
                    trials=trials)
        
        bestLoss = trials.best_trial['result']['loss']
        print(bestParams, bestLoss)
        prophetModel = Prophet(**bestParams)
        df = df[['ds','y']]
        prophetModel = prophetModel.fit(df)

        return bestLoss, bestParams, prophetModel
    
    def validate(self, params):
        if params['changepoint_prior_scale'] == 0:
            del params['changepoint_prior_scale']

In [5]:
class SarimaxModel():
    
    def __init__(self):
        self.p_values = np.arange(0, 2)
        self.d_values = np.arange(1, 2)
        self.q_values = np.arange(1, 4)
        self.P_values = np.arange(0, 2)
        self.D_values = np.arange(1, 2)
        self.Q_values = np.arange(0, 3)
        self.m_values = np.arange(7, 8)     
        
    def train(self, train, validation, arima_order, seasonalOrder):    
        try:          
            y_hat = validation.copy() 
            model = SARIMAX(train['sales'], order=arima_order, seasonal_order=seasonalOrder)        
            model_fit = model.fit()
            predict = model_fit.predict("2015-01-01", "2015-12-01", dynamic=True)
            y_hat['model_prediction'] = predict      

            evaluator = Evaluator(labelCol="sales", predictionCol="model_prediction")
            error = evaluator.Mape(y_hat) 
            
            #error = Mape(validation['sales'], y_hat.model_prediction)            
                        
            print('score: {0} model: {1}'.format(error, 'Sarimax'))
            return error, arima_order, seasonalOrder
        
        except Exception as e:                
            print(f"##### Skipped modelling with: {arima_order}, {seasonalOrder}\n")
            print(e)
            return -1, arima_order, seasonalOrder

    
    def evaluate(self, train, validation, p_values, d_values, q_values, P_values,
                 D_values, Q_values, m_values, parallel=True):    
        
        executor = Parallel(n_jobs=cpu_count()) 
        score = []

        if parallel==False:
            for p in p_values:
                for d in d_values:
                    for q in q_values:
                        for P in P_values:
                            for D in D_values:
                                for Q in Q_values:
                                    for m in m_values:        
                                        results = self.train(train, validation, (p,d,q), (P,D,Q,m)) 
                                        #print(results)
                                        score.append(results)                                        

        else:
            try:
                tasks = (delayed(self.train)(train, validation, (p,d,q), (P,D,Q,m)) for m in m_values for Q in Q_values for D in D_values for P in P_values for q in q_values for d in d_values for p in p_values)        
                results = executor(tasks)
                score.append(results)
            except Exception as e:
                print('Fatal Error....')
                print(e)

        return score
        
    def fit(self, data, labelCol):
        df = data.copy()
        data = HelperFunctions()
        train, validation = data.train_test_split(df, 2015)
        train = train[['ds',labelCol]]
        validation = validation[['ds',labelCol]]
        train.set_index('ds', inplace=True)
        validation.set_index('ds', inplace=True)
        train.index = pd.DatetimeIndex(train.index.values,
                                       freq=train.index.inferred_freq)
        validation.index = pd.DatetimeIndex(validation.index.values,
                                       freq=validation.index.inferred_freq)

        result = self.evaluate(train, validation, self.p_values, self.d_values, self.q_values,
                                         self.P_values, self.D_values, self.Q_values, self.m_values, False)
        
        scores=[]
        for tuple_list in result:            
            scores.append(tuple_list)

        scores.sort(key=lambda x: x[0])
        params = scores[0]
        
        df = df[['ds',labelCol]]
        df.set_index('ds', inplace=True)
        df.index = pd.DatetimeIndex(df.index.values,
                                       freq=df.index.inferred_freq)
        
        sarimaxModel = SARIMAX(df['sales'], order=params[1], seasonal_order=params[2])        
        sarimaxModel = sarimaxModel.fit()
        
        return params[0], (params[1], params[2]), sarimaxModel

In [27]:
class ModelSelector():
    def getModel(self, data):
        models = {}
        for key, value in data.items():
                                    
            result = Parallel(n_jobs=cpu_count(), prefer="threads")(delayed(self.parallelTraining)(model, value) 
                                                                     for model in ['prophet', 'sarimax'])                     
            
            bestLossProphet, bestParamsProphet, modelProphet = result[0]
            bestLossSarimax, bestParamsSarimax, modelSarimax = result[1]
            
            print(bestLossProphet, bestParamsProphet, modelProphet)
            print(bestLossSarimax, bestParamsSarimax, modelSarimax)            
            
            print("Best Loss Prophet: {0}".format(bestLossProphet))
            print("Best Loss Sarimax: {0}".format(bestLossSarimax))
            
            if bestLossProphet < bestLossSarimax:
                models[key] = ['Prophet', modelProphet]
            else:
                models[key] = ['Sarimax', modelSarimax]
            
        return models    
    
    def parallelTraining(self, model, data):
        if model == "prophet":
            prophet = ProphetModel()
            bestLossProphet, bestParamsProphet, prophetModel = prophet.fit(data, "sales")
            print(bestLossProphet, bestParamsProphet, prophetModel)
            return bestLossProphet, bestParamsProphet, prophetModel
        
        elif model == "sarimax":
            sarimax = SarimaxModel()
            bestLossSarimax, bestParamsSarimax, sarimaxModel = sarimax.fit(data, "sales")
            print(bestLossSarimax, bestParamsSarimax, sarimaxModel)            
            return bestLossSarimax, bestParamsSarimax, sarimaxModel        

In [24]:
train = pd.read_csv('train.csv', index_col = 0)
train['ds'] = pd.to_datetime(train[['year', 'month']].assign(day=1))
helper = HelperFunctions()
trainStores = helper.getStores(train)        

In [25]:
data = trainStores['CA_1']

In [29]:
sarimaxModel

In [20]:
class Driver():
    def main(self):
        train = pd.read_csv('train.csv', index_col = 0)
        test = pd.read_csv('test.csv', index_col = 0)
        
        train['ds'] = pd.to_datetime(train[['year', 'month']].assign(day=1))
        test['ds'] = pd.to_datetime(test[['year', 'month']].assign(day=1))
        
        helper = HelperFunctions()
        trainStores = helper.getStores(train)
        testStores = helper.getStores(test)        
        
        print("number of stores: {0}".format(len(trainStores)))                
        
        modelSelector = ModelSelector()
        return modelSelector.getModel(trainStores)    

In [28]:
if __name__ == "__main__":
    driver = Driver()
    models = driver.main()

number of stores: 10
  0%|                                                                            | 0/1 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.057845 seconds
INFO:hyperopt.tpe:TPE using 0 trials
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 37.


score: 0.021063141003058453 model: Sarimax                                                                             
score: 0.02454867902793963 model: Prophet                                                                              
100%|█████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.95s/trial, best loss: 0.02454867902793963]

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



{'changepoint_prior_scale': 2, 'n_changepoints': 8} 0.02454867902793963
score: 0.017846001294108318 model: Sarimax
0.02454867902793963 {'changepoint_prior_scale': 2, 'n_changepoints': 8} <fbprophet.forecaster.Prophet object at 0x0000026F8CBAD588>
score: 0.01693326699881369 model: Sarimax
score: 0.017106446850248588 model: Sarimax
score: 0.017152597669482805 model: Sarimax
score: 0.01736799063541656 model: Sarimax
score: 0.022126131915158175 model: Sarimax
score: 0.018010692656682783 model: Sarimax
score: 0.017097676392160158 model: Sarimax
score: 0.017257514558625446 model: Sarimax
score: 0.020237665848885162 model: Sarimax
score: 0.017248455572712606 model: Sarimax
score: 0.023052568318454956 model: Sarimax
score: 0.018244945712127047 model: Sarimax
score: 0.016976716649639057 model: Sarimax
score: 0.017025263780505073 model: Sarimax
score: 0.020192452647799574 model: Sarimax
score: 0.017094092569126657 model: Sarimax
score: 0.02248717665328924 model: Sarimax
score: 0.018038730969357

...


{'changepoint_prior_scale': 4, 'n_changepoints': 1} 0.06896345369553342
0.06896345369553342 {'changepoint_prior_scale': 4, 'n_changepoints': 1} <fbprophet.forecaster.Prophet object at 0x0000026F8CB8B9C8>
score: 0.009665502236687372 model: Sarimax
score: 0.009862656441650741 model: Sarimax
score: 0.010319501748077132 model: Sarimax
score: 0.012835405491561017 model: Sarimax
score: 0.012020444490896736 model: Sarimax
score: 0.009885871559972274 model: Sarimax
score: 0.009984581491099225 model: Sarimax
score: 0.010030224164756773 model: Sarimax
score: 0.010582598451312819 model: Sarimax
score: 0.012437902274211243 model: Sarimax
score: 0.011708718971717725 model: Sarimax
score: 0.010347637362975256 model: Sarimax
score: 0.010070820516472446 model: Sarimax
score: 0.0099479331014569 model: Sarimax
score: 0.010386475009679064 model: Sarimax
score: 0.01228181013545345 model: Sarimax
score: 0.011683149743753854 model: Sarimax
score: 0.009699669261471883 model: Sarimax
score: 0.009973986205701

INFO:hyperopt.tpe:build_posterior_wrapper took 0.000000 seconds
INFO:hyperopt.tpe:TPE using 0 trials
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 37.


score: 0.010584787511429383 model: Sarimax                                                                             
score: 0.027690918222007147 model: Prophet                                                                             
100%|████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.90s/trial, best loss: 0.027690918222007147]

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



{'changepoint_prior_scale': 5, 'n_changepoints': 5} 0.027690918222007147
0.027690918222007147 {'changepoint_prior_scale': 5, 'n_changepoints': 5} <fbprophet.forecaster.Prophet object at 0x0000026F8DC8D508>
score: 0.00981968561773722 model: Sarimax
score: 0.02983661433728907 model: Sarimax
score: 0.009625526944228724 model: Sarimax
score: 0.009553168678074951 model: Sarimax
score: 0.03284287077557983 model: Sarimax
score: 0.02159775693153231 model: Sarimax
score: 0.014437705841497742 model: Sarimax
score: 0.022671688448718006 model: Sarimax
score: 0.00938296588720383 model: Sarimax
score: 0.013435968790089023 model: Sarimax
score: 0.02083641669350209 model: Sarimax
score: 0.023637528161939907 model: Sarimax
score: 0.024128144404538435 model: Sarimax
score: 0.023325178936753763 model: Sarimax
score: 0.02370083072092372 model: Sarimax
score: 0.024310394449641295 model: Sarimax
score: 0.022346908291258964 model: Sarimax
score: 0.009735952895920774 model: Sarimax
score: 0.01024040989420833

In [17]:
models[0][0]

0.019889788737184385

In [18]:
b , c, d = models[0]

In [13]:
mo

(0.01693326699881369,
 ((0, 1, 1), (0, 1, 2, 7)),
 <statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper at 0x26f8afaed88>)